In [2]:
#set style for plots
sns.set_style('white')
sns.despine()
#plt.style.use('seaborn-whitegrid')
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

<Figure size 640x480 with 0 Axes>

In [3]:
df= pd.read_csv('../dataset/final.csv')
df.head()

,sentiment,tweet
0,0.0,#AAPL:The 10 best Steve Jobs emails ever...htt...
1,0.0,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2,0.0,My cat only chews @apple cords. Such an #Apple...
3,0.0,I agree with @jimcramer that the #IndividualIn...
4,0.0,Nobody expects the Spanish Inquisition #AAPL


In [4]:
df.iloc[0].tweet

'#AAPL:The 10 best Steve Jobs emails ever...http://t.co/82G1kL94tx'

### Deep Data Cleaning

In [5]:
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [6]:
texts_new = []
for t in df.tweet:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(t)))))

In [7]:
df['clean_text']=texts_new
df.head()

,sentiment,tweet,clean_text
0,0.0,#AAPL:The 10 best Steve Jobs emails ever...htt...,aaplthe 10 best steve jobs emails ever
1,0.0,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...,rt why aapl stock had a miniflash crash today ...
2,0.0,My cat only chews @apple cords. Such an #Apple...,my cat only chews cords such an applesnob
3,0.0,I agree with @jimcramer that the #IndividualIn...,i agree with that the individualinvestor shoul...
4,0.0,Nobody expects the Spanish Inquisition #AAPL,nobody expects the spanish inquisition aapl


In [8]:
text_len = []
for text in df.clean_text:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [9]:
df['text_len']=text_len
df[['sentiment', 'clean_text']].to_csv('../dataset/data.csv', index=False)

In [10]:
df.iloc[0].clean_text

'aaplthe 10 best steve jobs emails ever'

In [11]:
df.head()

,sentiment,tweet,clean_text,text_len
0,0.0,#AAPL:The 10 best Steve Jobs emails ever...htt...,aaplthe 10 best steve jobs emails ever,7
1,0.0,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...,rt why aapl stock had a miniflash crash today ...,11
2,0.0,My cat only chews @apple cords. Such an #Apple...,my cat only chews cords such an applesnob,8
3,0.0,I agree with @jimcramer that the #IndividualIn...,i agree with that the individualinvestor shoul...,21
4,0.0,Nobody expects the Spanish Inquisition #AAPL,nobody expects the spanish inquisition aapl,6


### Balancing the dataset

In [12]:
df['sentiment'].value_counts()

sentiment
 0.0    3676
-1.0    2235
 1.0     704
Name: count, dtype: int64

In [13]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['clean_text']).reshape(-1, 1), np.array(df['sentiment']).reshape(-1, 1))
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['clean_text', 'sentiment'])
train_os['sentiment'].value_counts()

sentiment
 0.0    3676
 1.0    3676
-1.0    3676
Name: count, dtype: int64

In [3]:
train_os = train_os[train_os.clean_text != '']
train_os.to_csv('../dataset/data.csv', index=None)

NameError: name 'train_os' is not defined

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(dataset['clean_text'], dataset['sentiment'], test_size=0.1, random_state=42)

print(X_train)

['give me this pictwittercomhlvxntisug', 'wouldnt be a bad idea to bring back the tap to tweet function guys', 'rt insanely great news teams up with for iads programmatic launch', 'have patented an iphone drop protection mechanism is the is end of crackedscreens', 'tell to update their events page theres no listing for your presentation', 'apples iphone 6 plus amazingly captures 41 of the phablet market right out of the gate aapl aapl', 'is apple hedging its bets with another small iphone aapl', 'ironic that were at a pr event mp15 talking about companies that dont need more people talking about them', 'i know this is going sound simple but with ios7 without entering the password you have a flashlight genius simple wow', 'all hyperbole aside this article as well as are simply amazing arent they via', 'what to expect in 2015', 'ipod replaced if i never have to set foot in an store again the world will be a better place', 'afterhours buzz aapl bobe amp more check out which companies are 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#pre processing
import regex as re
import string
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

#
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel, TFAutoModelForSequenceClassification

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

2024-02-04 10:19:05.550957: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 10:19:05.576732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 10:19:05.576761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 10:19:05.577436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 10:19:05.581708: I tensorflow/core/platform/cpu_feature_guar

### train-validation dataset split

In [1]:
from datasets import load_dataset

dataset = load_dataset("csv", split='train', data_files="../dataset/data.csv")

/mnt/Data/projects/roberta/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Tokenizing

In [12]:
from transformers import AutoTokenizer, AutoConfig
tokenizer= AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [3]:
def tokenize(examples):
    token = tokenizer(examples['clean_text'], truncation=True, padding="max_length")
    return token

tokenized_dataset = dataset.map(tokenize)

In [4]:
tokenized_dataset

Dataset({
    features: ['clean_text', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 11009
})

### RoBERTa Sentiment Analysis

In [5]:
small_train_dataset = tokenized_dataset.shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_dataset

print(small_train_dataset)
print(small_train_dataset['input_ids'])

Dataset({
    features: ['clean_text', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 1000
})
[[0, 4892, 3275, 13, 24, 438, 1553, 1551, 165, 13, 9311, 39535, 3946, 5032, 1792, 12478, 11, 95, 132, 722, 2446, 2], [0, 808, 330, 24, 95, 48288, 1397, 600, 2], [0, 118, 393, 5324, 141, 205, 5, 1437, 28778, 1264, 195, 2280, 21, 181, 3252, 40123, 605, 7915, 175, 288, 329, 401, 329, 417, 398, 506, 8307, 330, 2], [0, 27333, 7, 244, 1594, 1717, 304, 761, 1617, 634, 15162, 609, 16, 481, 8, 1769, 385, 119, 19, 2], [0, 43395, 110, 92, 1437, 28778, 1264, 64, 342, 110, 3599, 23, 810, 1241, 13468, 1846, 15406, 2], [0, 28778, 1264, 195, 438, 21, 3518, 7, 28, 6162, 53, 63, 5, 276, 425, 25, 97, 1437, 28778, 6909, 47, 17534, 162, 2], [0, 10932, 24, 8349, 16, 10, 2793, 177, 190, 4996, 16, 205, 2], [0, 118, 4940, 2854, 2], [0, 2794, 36140, 1434, 786, 405, 16454, 930, 8, 46405, 1137, 106, 59, 24, 1241, 15162, 930, 2], [0, 27326, 16, 8232, 62, 7, 592, 433, 15162, 16, 5947, 10, 592, 433, 25259, 11, 8

### Building Model

In [19]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TFTrainingArguments
training_args = TFTrainingArguments("my_model")
from transformers import TFTrainer
trainer = TFTrainer(model=model, args=training_args, train_dataset=full_train_dataset, tokenizer=tokenizer)


ImportError: cannot import name 'TFTrainer' from 'transformers' (/mnt/Data/projects/roberta/lib/python3.11/site-packages/transformers/__init__.py)

In [9]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [28]:

# Freeze pre-trained layers
# for layer in model.layers:
#     print(layer)
#     layer.trainable = False
inputs = preprocess("Apple is bad")
encoded_input = tokenizer(inputs, return_tensors='tf')

print(encoded_input)
output = model(encoded_input)
# Add new layers for your specific task

# print(model.summary())
# model.build(X_train.shape)

# print(model.summary())

# print(roberta_model.summary())
# Compile the model
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
# model.fit(X_train, epochs=10, validation_data=X_valid)


{'input_ids': tensor([[    0, 20770,    16,  1099,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


ValueError: Exception encountered when calling layer 'tf_roberta_for_sequence_classification_1' (type TFRobertaForSequenceClassification).

Data of type <class 'torch.Tensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_roberta_for_sequence_classification_1' (type TFRobertaForSequenceClassification):
  • input_ids={'input_ids': 'tensor([[    0, 20770,    16,  1099,     2]])', 'attention_mask': 'tensor([[1, 1, 1, 1, 1]])'}
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • labels=None
  • training=False